# Scraping Indeed sivustosta 24.2 

# Johdato

Tämä on osa projektityötä "2022 kesätyöanalyysi indeed-sivustosta: Python-työpaikkoja". Tässä ipynb-tiedostosta käytetään web scraping datankeräämiseen. 

In [2]:
import requests
from bs4 import BeautifulSoup
import re 
import pandas as pd
import numpy as np 

/home/degnaiyu/.local/lib/python3.8/site-packages/pandas/compat/_optional.py:149: UserWarning: Pandas requires version '1.3.1' or newer of 'bottleneck' (version '1.2.1' currently installed).
  warnings.warn(msg, UserWarning)


Base link: https://fi.indeed.com/

In [3]:
baselink= 'https://fi.indeed.com'

Haulla 'Kesätyö' saatiin seuraava:

In [4]:
url= 'https://fi.indeed.com/jobs?q=Kesätyö&start=0'

Sivusto haulla 'Kesätyö': 

In [5]:
page=requests.get(url) #sivusto 

## Sivuston html-script

In [6]:
soup=BeautifulSoup(page.content,'html.parser' )

## Sivuston tietoaukeama 

Tässä on kaikki olennaiset tiedot yhdestä aukeamasta

In [7]:
maincontent= soup.find('tbody', id= 'resultsBodyContent')

### Sivujen numerot listana

In [8]:
sivunumerot= list(range(10, 340, 10 ))     #ensimmäinen sivu ei huomioida

## Yhden sivun analysointi

Analysoidaan yhtä sivua 

In [9]:
mozaiczone= maincontent.find('div' ,class_= 'mosaic-zone', id='mosaic-zone-jobcards' )   #tässä on kaikki työesitteet 
jobcards= mozaiczone.find_all('a',rel="nofollow",  target='_blank')   #tässä on kaikki linkkien esitteet  

Linkit kesätyöpaikkojen sivustoihin:

In [10]:
jobcardlinklist=[]    

In [11]:
for job in jobcards:
    jobcardlinklist.append(baselink+ job['href'])

Näytetään ensimmäiset linkit:

In [12]:
jobcardlinklist

['https://fi.indeed.com/rc/clk?jk=521be086e6d00ea2&fccid=264e4b7d5396a070&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=3c2e84186be30099&fccid=a5aea01671306f7a&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=032e3967eebf9fa6&fccid=0ce9a45d8b4ba0dc&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=a25e48332f4bc442&fccid=b3d8921676c3d832&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=ced20c43f3e5875d&fccid=6fd093bf1614588a&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=b51b4a93b4b6232f&fccid=e16a8bcfd3cd2559&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=158f8e5a963b1c11&fccid=e4800948fed13758&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=2ac37ec906323673&fccid=d29a99d6688ab1b2&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=6fb17225b2c0a8a8&fccid=289f8d669b92b0c1&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=501bffc9396e4a0e&fccid=c91e9d2c7be1c3e0&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=31c7c7a632161cd8&fccid=e16a8bcfd3cd2559&vjs=3',
 'https://fi.indeed.com/rc/clk?jk=24cce3a6f90b1c96&fccid=c3b919e6b9bfa39e&vjs=3',
 'https://fi.ind

## Funktio työpaikan sivustolle:

Funktio palauttaa työpaikan sivun tekstiosan.  

In [13]:
#testattu

def seulonta(link):
    webpage= requests.get(link) 
    bsoup= BeautifulSoup(webpage.content, 'html.parser')
    #työpaikan esite
    try: 
        jobdescriptiontext= bsoup.find('div', id="jobDescriptionText" ).text 
    except:
        return np.nan, np.nan, np.nan, np.nan
    #työpaikan nimike
    nimike= bsoup.find('h1', class_='icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title' ).text 
    
    
    #tässä on tekstipätkä, jossa on yrityksen nimi, arvostelun määrä (epäolennainen)
    # ja yrityksen sijaintikunta  
    companyinfo= bsoup.find('div', class_= "icl-u-xs-mt--xs icl-u-textColor--secondary jobsearch-JobInfoHeader-subtitle jobsearch-DesktopStickyContainer-subtitle").text 
    
    
    #poistetaan arvostelun määrä osio 
    if re.search(r'\d+ arvostelua', companyinfo): 
        companyinfo= re.sub(r'\d+ arvostelua', ',', companyinfo)
        
        companyinfo= companyinfo.split(',')  # companyinfo on nyt listana


        yritysnimi= companyinfo[0]
        paikkakunta= companyinfo[-1]
    
    elif 'arvostelua' not in companyinfo: 
        try: 
            yritysnimi, = re.findall(r'([a-zåäöA-ZÅÄÖ].+)[A-ZÅÄÖ]', companyinfo)
            paikkakunta = companyinfo.replace(yritysnimi, '')
        except: 
            yritysnimi =np.nan 
            paikkakunta=np.nan  
        
    else: 
        yritysnimi= np.nan 
        paikkakunta= np.nan
    
    return nimike, yritysnimi, paikkakunta, jobdescriptiontext
    
    

In [14]:
seulonta(jobcardlinklist[2])   #esimerkki 

('Myyjä (kesätyöntekijä), Kankaanpää',
 'Musti ja Mirri Oy',
 'Helsinki',
 'Myyjä Kankaanpään Musti ja Mirri myymälään (kesätyö)\nTervetuloa Mustilaumaan! Olemme Pohjoismaiden suurin ja ammattitaitoisin lemmikkitarvikeketju, ja tunnettuja loistavasta asiakaspalvelusta. Me teemme lemmikkien ja niiden perheiden elämästä helpompaa, turvallisempaa ja hauskempaa.\nMillaista on myyjän työ Mustissa ja Mirrissä?\n"Musti ja Mirri ei ole kuin mikä tahansa kauppa, vaan eläinihmisten kohtaamispaikka. Asiakkaiden pulmien ratkominen on äärimmäisen palkitsevaa. Minulla on ilo ja kunnia olla tukena nelijalkaisten asiakkaideni arjessa – sillä jokainen kuono on loistavan palvelun arvoinen.\nVoin päästää luovuuteni valloilleen luomalla uusia esillepanoja tai päivittämällä myymälän sometilejä. Mikäli haluan skipata kuntosalin tällä(kin) viikolla, nappaan vastuun isoimpien ruokasäkkien hyllyttämisestä. Kun haluan haastaa itseäni, otan hoitaakseni myymälätapahtuman järjestämisen. Mustissa ja Mirrissä mahdol

## Tiedon tallentaminen DataFrame-taulukkoon 

Taulukko:

In [15]:
frame= {'Työpaikan nimike':[], 'Yritysnimi': [], 'Sijaintikunta': [], 'Seloste': []}

In [16]:
for link in jobcardlinklist:
    nimike, yritysnimi, paikkakunta, jobdescriptiontext = seulonta(link)
    frame['Työpaikan nimike'].append(nimike)
    frame['Yritysnimi'].append(yritysnimi)
    frame['Sijaintikunta'].append(paikkakunta)
    frame['Seloste'].append(jobdescriptiontext)
    

In [17]:
pd.DataFrame(frame).head()

,Työpaikan nimike,Yritysnimi,Sijaintikunta,Seloste
0,Pizzeriatyöntekijä Kesätyö 2022 Rax Pizzabuffe...,Restel Ravintolat Oy,Helsinki,"Oletko reipas ja vastuuntuntoinen tyyppi, joka..."
1,Kesärekry - Kesätyöntekijöitä Customer Service...,JYSK,Vantaa,Yrityksen kuvaus\nHaemme kesätyöntekijöitä 30-...
2,"Myyjä (kesätyöntekijä), Kankaanpää",Musti ja Mirri Oy,Helsinki,Myyjä Kankaanpään Musti ja Mirri myymälään (ke...
3,Jäätelönmyyjä,Finncafe,Helsinki,We are looking for people who can sell ice cre...
4,"Kesätyö, 16-17-vuotiaat",Helsingin Kaupunki,Helsinki,Hei sinä 16-17-vuotias nuori!\n\n\nKaipaatko k...


## Monen sivun analysointi

Tässä osassa toistetaan edelliset vaiheet. Tarkoituksena on analysoida kaikki sivut, jotka liittyvät kesätyöpaikkaan. 

In [18]:
for sivunumero in sivunumerot:
    #muutetaan sivunumeroa 
    url = re.sub('\d+$', str(sivunumero), url)   
    
    #päästään sivun html-scriptiin
    page=requests.get(url)
    soup= BeautifulSoup(page.content, 'html.parser')
    
    #sisältö-alue 
    maincontent= soup.find('tbody', id= 'resultsBodyContent')
    
    #työpaikkojen esitteet 
    mozaiczone= maincontent.find('div' ,class_= 'mosaic-zone', id='mosaic-zone-jobcards' )   #tässä on kaikki työesitteet 
    jobcards= mozaiczone.find_all('a',rel="nofollow",  target='_blank')   #tässä on kaikki linkkien esitteet  

    
    jobcardlinklist=[] #nollataan lista 
    
    #tallennetaan sivuston linkit listaan 
    for job in jobcards:
        jobcardlinklist.append(baselink+ job['href'])
        
        
    for link in jobcardlinklist:
        nimike, yritysnimi, paikkakunta, jobdescriptiontext = seulonta(link)
        frame['Työpaikan nimike'].append(nimike)
        frame['Yritysnimi'].append(yritysnimi)
        frame['Sijaintikunta'].append(paikkakunta)
        frame['Seloste'].append(jobdescriptiontext)





AttributeError: 'NoneType' object has no attribute 'find'

Näköjään esiintyi recaptcha-botti. Jatketaan edellistä seulontaa. 

In [30]:
for sivunumero in range(170,340, 10 ): 
    #muutetaan sivunumeroa 
    url = re.sub('\d+$', str(sivunumero), url)   
    
    #päästään sivun html-scriptiin
    page=requests.get(url)
    soup= BeautifulSoup(page.content, 'html.parser')
    
    #sisältö-alue 
    maincontent= soup.find('tbody', id= 'resultsBodyContent')
    
    #työpaikkojen esitteet 
    mozaiczone= maincontent.find('div' ,class_= 'mosaic-zone', id='mosaic-zone-jobcards' )   #tässä on kaikki työesitteet 
    jobcards= mozaiczone.find_all('a',rel="nofollow",  target='_blank')   #tässä on kaikki linkkien esitteet  

    
    jobcardlinklist=[] #nollataan lista 
    
    #tallennetaan sivuston linkit listaan 
    for job in jobcards:
        jobcardlinklist.append(baselink+ job['href'])
        
        
    for link in jobcardlinklist:
        nimike, yritysnimi, paikkakunta, jobdescriptiontext = seulonta(link)
        frame['Työpaikan nimike'].append(nimike)
        frame['Yritysnimi'].append(yritysnimi)
        frame['Sijaintikunta'].append(paikkakunta)
        frame['Seloste'].append(jobdescriptiontext)



In [33]:
sivunumero

330

Sivunumero on kunnossa. Lukeminen onnistui 

## Data muunnetaan taulukoksi pd.DataFrame

In [34]:
dataframe= pd.DataFrame(frame)

Tallennetaan taulukko json tiedostona: 

In [37]:
#data=dataframe.to_json('./scrapingresult.json')    